In [ ]:
# default_exp quantification

# Quantification

> Functions related to quantification

In [ ]:
#hide
from nbdev.showdoc import *

## Label-free quantification

Algorithms related to label-free quantifications are motivated by the [MaxLFQ paper](https://doi.org/10.1074/mcp.m113.031591). The main goal is to derive relative protein intensities that can be used for downstream analyses. In a first step, constant normalization coefficients are derived for each run. In a second step, pseudointensities are derived for each protein, such that differing conditions can be compared.

## Delayed Normalization

Delayed normalization describes the process of normalizing the differences that occur from prefractionation as well as from sample handling. For each sample, a constant scaling factor is derived by minimizing the term
$$H(\vec{N}) = \sum_{P \in peptides} \sum_{A,B \in sample pairs} |\frac{I(N_A, P, A)}{I(N_B, P, B)}|, $$
with peptide intensities $I$, which are determined by the peptide $P$ the sample $A$ or $B$ and the normalization factors $N_A$, $N_B$. In principle H(N) quantifies the variation of peptides over the samples. Minimizing this variation gives appropriate scaling factors under the assumption that most peptides do not change between the samples. Peptide intensities for fractionated samples are described as the sum of the intensities over the fractions, with fraction-specific normalization factors. Therefore, calculation of the summed intensities is *delayed* until the normalization is finished.

### In Silico Test data

To test the delayed normalization approach we create an in silico test dataset with a known ground truth. We therefore know, which systematic changes are between the samples and we employ different solvers to recover the normalization parameters. 

In [ ]:
#export
import random
import numpy as np
import logging

def gaussian(mu, sigma, grid):
    """
    Gaussian function
    """
    norm = 0.3989422804014327 / sigma
    return norm * np.exp(-0.5 * ((grid - mu) / sigma) ** 2)


def return_elution_profile(timepoint, sigma, n_runs):
    """
    Returns a gaussian elution profile for a given timepoint
    """
    return gaussian(timepoint, sigma, np.arange(0, n_runs))


def simulate_sample_profiles(n_peptides, n_runs, n_samples, threshold=0.2, use_noise=True):
    """
    Generate random profiles to serve as test_data
    """
    abundances = np.random.rand(n_peptides)*10e7

    true_normalization = np.random.normal(loc=1, scale=0.1, size=(n_runs, n_samples))

    true_normalization[true_normalization<0] = 0

    true_normalization = true_normalization/np.max(true_normalization)

    maxvals = np.max(true_normalization, axis=1)

    elution_timepoints = random.choices(list(range(n_runs)), k=n_peptides)

    profiles = np.empty((n_runs, n_samples, n_peptides))
    profiles[:] = np.nan

    for i in range(n_peptides):

        elution_timepoint = elution_timepoints[i]
        abundance = abundances[i]

        profile = return_elution_profile(elution_timepoint, 1, n_runs)
        profile = profile/np.max(profile)
        profile = profile * abundance
        elution_profiles = np.tile(profile, (n_samples, 1)).T

        # Add Gaussian Noise
        if use_noise:
            noise = np.random.normal(1, 0.2, elution_profiles.shape)
            noisy_profile = noise * elution_profiles
        else:
            noisy_profile = elution_profiles

        normalized_profile = noisy_profile * true_normalization

        normalized_profile[normalized_profile < threshold] = 0
        normalized_profile[normalized_profile == 0] = np.nan


        profiles[:,:,i] = normalized_profile

    return profiles, true_normalization

## Delayed Normalization

In [ ]:
#export
from numba import njit, prange

@njit
def get_peptide_error(profile, normalization):

    pep_ints = np.zeros(profile.shape[1])

    normalized_profile = profile*normalization

    for i in range(len(pep_ints)):
        pep_ints[i] = np.nansum(normalized_profile[:,i])

    pep_ints = pep_ints[pep_ints>0]

    # Loop through all combinations
    n = len(pep_ints)

    error = 0
    for i in range(n):
        for j in range(i+1,n):
            error += np.abs(np.log(pep_ints[i]/pep_ints[j]))**2

    return error


@njit(parallel=True)
def get_total_error_parallel(normalization, profiles):

    normalization = normalization.reshape(profiles.shape[:2])

    total_error = 0

    for index in prange(profiles.shape[2]):
        total_error += get_peptide_error(profiles[:,:, index], normalization)

    return total_error


def get_total_error(normalization, profiles):

    normalization = normalization.reshape(profiles.shape[:2])

    total_error = 0

    for index in range(profiles.shape[2]):
        total_error += get_peptide_error(profiles[:,:, index], normalization)

    return total_error

In [ ]:
#export
from scipy.optimize import minimize
import pandas as pd
import numpy as np

def normalize_experiment_SLSQP(profiles):
    """
    Calculate normalization with SLSQP approach
    """
    x0 = np.ones(profiles.shape[0] * profiles.shape[1])
    bounds = [(0.1, 1) for _ in x0]
    res = minimize(get_total_error, args = profiles , x0 = x0*0.5, bounds=bounds, method='SLSQP', options={'disp': False} )

    solution = res.x/np.max(res.x)
    solution = solution.reshape(profiles.shape[:2])

    return solution

def normalize_experiment_BFGS(profiles):
    """
    Calculate normalization with BFGS approach
    """
    x0 = np.ones(profiles.shape[0] * profiles.shape[1])
    bounds = [(0.1, 1) for _ in x0]
    res = minimize(get_total_error, args = profiles , x0 = x0*0.5, bounds=bounds, method='L-BFGS-B', options={'disp': False} )

    solution = res.x/np.max(res.x)
    solution = solution.reshape(profiles.shape[:2])

    return solution

def delayed_normalization(df, field='int_sum', minimum_occurence=None):
    """
    Returns normalization for given peptide intensities
    """
    files = np.sort(df['shortname'].unique()).tolist()
    n_files = len(files)

    if 'fraction' not in df.keys():
        df['fraction'] = [1 for x in range(len(df.index))]
    
    fractions = np.sort(df['fraction'].unique()).tolist()

    n_fractions = len(fractions)

    df_max = df.groupby(['precursor','fraction','shortname'])[field].max() #Maximum per fraction

    prec_count = df_max.index.get_level_values('precursor').value_counts()

    if not minimum_occurence:
        minimum_occurence = np.percentile(prec_count[prec_count>1].values, 75) #Take the 25% best datapoints
        logging.info('Setting minimum occurence to {}'.format(minimum_occurence))

    shared_precs = prec_count[prec_count >= minimum_occurence]
    precs = prec_count[prec_count > minimum_occurence].index.tolist()

    n_profiles = len(precs)

    selected_precs = df_max.loc[precs]
    selected_precs = selected_precs.reset_index()

    profiles = np.empty((n_fractions, n_files, n_profiles))
    profiles[:] = np.nan

    #get dictionaries
    fraction_dict = {_:i for i,_ in enumerate(fractions)}
    filename_dict = {_:i for i,_ in enumerate(files)}
    precursor_dict = {_:i for i,_ in enumerate(precs)}

    prec_id = [precursor_dict[_] for _ in selected_precs['precursor']]
    frac_id = [fraction_dict[_] for _ in selected_precs['fraction']]
    file_id = [filename_dict[_] for _ in selected_precs['shortname']]

    profiles[frac_id,file_id, prec_id] = selected_precs[field]

    try:
        normalization = normalize_experiment_SLSQP(profiles)
    except ValueError: # SLSQP error in scipy https://github.com/scipy/scipy/issues/11403
        logging.info('Normalization with SLSQP failed. Trying BFGS')
        normalization = normalize_experiment_BFGS(profiles)


    #intensity normalization: total intensity to remain unchanged

    df[field+'_dn'] = df[field]*normalization[[fraction_dict[_] for _ in df['fraction']], [filename_dict[_] for _ in df['shortname']]]
    df[field+'_dn'] *= df[field].sum()/df[field+'_dn'].sum()

    # else:
    #     logging.info('No fractions present. Skipping delayed normalization.')
    #     normalization = None

    return df, normalization

In [ ]:
sample_data = {}

sample_data['precursor'] = ['Prec_1'] * 6 + ['Prec_2'] * 6 + ['Prec_3'] * 6
sample_data['fraction'] = [1,1,2]*6
sample_data['shortname'] = ['A','A','A', 'B','B','B'] * 3
sample_data['int_sum'] = [0.6, 0.8, 0.6, 1.2, 1.6, 1.2] * 3

test_df = pd.DataFrame(sample_data)
test_df, normalization = delayed_normalization(test_df, field='int_sum', minimum_occurence=2)

display(pd.DataFrame(normalization))
display(test_df.head(6))


/opt/anaconda3/lib/python3.8/site-packages/scipy/optimize/optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


,0,1
0,0.957777,0.480980
1,1.000000,0.497222


,precursor,fraction,shortname,int_sum,int_sum_dn
0,Prec_1,1,A,0.6,0.887676
1,Prec_1,1,A,0.8,1.183568
2,Prec_1,2,A,0.6,0.926809
3,Prec_1,1,B,1.2,0.891552
4,Prec_1,1,B,1.6,1.188736
5,Prec_1,2,B,1.2,0.921659


## Benchmarking different optimiziers
The normalization step is in principle a quadratic minimization of the normalization factors. Such minimization problems can be solved in various ways and a variety of approaches are realized in python community packages. We compare different solvers using our benchmarking set and uncover substantial differences in precision and runtime. We observe that the *Sequential Least Squares Quadratic Programming* (SLSQP) approach is a robust solution in our benchmarking, which gives substantial speed improvements. 

In [ ]:
from scipy.optimize import minimize
from time import time
from scipy.optimize import least_squares
import pandas as pd


n_peptides = 100
n_runs = 10
n_samples = 3

profiles, true_normalization = simulate_sample_profiles(n_peptides, n_runs, n_samples)

methods = ['L-BFGS-B', 'TNC', 'SLSQP','trf']

results = []

for method in methods:
    
    start = time()
    
    if method in ['trf']:
        x0 = np.ones(profiles.shape[0] * profiles.shape[1])
        bounds = (x0*0.1, x0)
        res = least_squares(get_total_error, args = [profiles], bounds = bounds, x0 = x0*0.5, verbose=0, method = method)

    else:
        x0 = np.ones(profiles.shape[0] * profiles.shape[1])
        bounds = [(0.1, 1) for _ in x0]
        res = minimize(get_total_error, args = profiles , x0 = x0*0.5, bounds=bounds, method=method)

    solution = res.x/np.max(res.x)
    solution = solution.reshape(profiles.shape[:2])
    
    end = time()
    
    time_elapsed_min = (end-start)/60

    optimality = get_total_error(solution, profiles) /get_total_error(x0, profiles)
    optimality_ = get_total_error(solution, profiles) / get_total_error(true_normalization, profiles)
    
    results.append((method, time_elapsed_min, optimality, optimality_))
    
pd.DataFrame(results, columns=['Method', 'Time Elapsed (min)','Error / Baseline Error','Error / Ground Truth'])

C:\ProgramData\Anaconda3\envs\alphapept\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


,Method,Time Elapsed (min),Error / Baseline Error,Error / Ground Truth
0,L-BFGS-B,0.038645,0.613492,0.336389
1,TNC,0.019305,0.638834,0.350285
2,SLSQP,0.003049,0.613492,0.336389
3,trf,0.184128,0.615414,0.337443


## Implementation

## Constructing protein intensity profiles
Protein intensity profiles are constructed for each protein individually. All possible protein fold changes between the samples are derived from the median peptide fold changes. Subsequently, pseudointensities are chosen such that the fold changes between the pseudointensities ideally reconstruct the actually observed fold changes. Similar to the delayed normalization, this is formulated as a quadratic minimization, which we solve with the SLSQP solver.

## Test Data

In [ ]:
import numpy as np
import string
from time import time
import pandas as pd

def generate_dummy_data(n_sequences, n_samples, noise=True, remove = True, peptide_ratio = True, abundance=True, signal_level=100, noise_divider=10, keep=0.8):

    species = ['P'+str(_) for _ in range(1,n_sequences+1)]
    sample = [string.ascii_uppercase[_%26]+str(_//26) for _ in range(n_samples)]
    
    if peptide_ratio:
        peptide_ratio = np.random.rand(n_sequences)
        peptide_ratio = peptide_ratio/np.sum(peptide_ratio)
    else:
        peptide_ratio = np.ones(n_sequences)

    if abundance:
        abundance_profile = np.random.rand(n_samples,1)
    else:
        abundance_profile = np.ones((n_samples,1))

    original_signal = np.ones((n_samples, n_sequences))

    noise_sim = (np.random.rand(n_samples, n_sequences)-0.5)/noise_divider

    if noise:
        noisy_signal = original_signal+noise_sim
        noisy_signal = noisy_signal*signal_level*peptide_ratio*abundance_profile
    else:
        noisy_signal = original_signal*signal_level*peptide_ratio*abundance_profile

    if remove:
        #Remove points
        keep_probability = keep #keep 60% of the points 
        to_remove = np.random.rand(n_samples, n_sequences)
        to_remove = to_remove>=keep_probability

        dummy_data = noisy_signal.copy()

        dummy_data[to_remove] = 0

    else:
        dummy_data = noisy_signal

            
    dummy_data = pd.DataFrame(dummy_data, index = sample, columns = species).T
    
    ground_truth = abundance_profile.flatten()
    ground_truth = ground_truth/np.max(ground_truth)
        
    return dummy_data, sample, ground_truth

## Determine pair-wise intenisty ratios
The pair-wise protein ratios are determined from the median peptide ratio.

In [ ]:
#export
from numba import njit

@njit
def get_protein_ratios(signal, column_combinations, minimum_ratios = 1):
    n_samples = signal.shape[1]
    ratios = np.empty((n_samples, n_samples))
    ratios[:] = np.nan

    for element in column_combinations:
        i = element[0]
        j = element[1]

        ratio = signal[:,j] / signal[:,i]

        non_nan = np.sum(~np.isnan(ratio))

        if non_nan >= minimum_ratios:
            ratio_median = np.nanmedian(ratio)
        else:
            ratio_median = np.nan

        ratios[j,i] = ratio_median

    return ratios

## Error Function
The error function evaluates the difference between the actually observed fold change and the fold change that is derived from the pseudointensities.

In [ ]:
#export
@njit
def triangle_error(normalization, ratios):
    int_matrix = np.repeat(normalization, len(normalization)).reshape((len(normalization), len(normalization))).transpose()
    x = (np.log(ratios) - np.log(int_matrix.T) + np.log(int_matrix))**2

    return np.nansum(x)

## Different ways to solve
Different quadratic solving strategies are tested.

In [ ]:
#export
## L-BFGS-B


from scipy.optimize import minimize, least_squares

# LFBGSB

def solve_profile_LFBGSB(ratios):
    x0 = np.ones(ratios.shape[1])
    bounds = [(min(np.nanmin(ratios), 1/np.nanmax(ratios)), 1) for _ in x0]
    res_wrapped = minimize(triangle_error, args = ratios , x0 = x0, bounds=bounds, method = 'L-BFGS-B')
    solution = res_wrapped.x
    solution = solution/np.max(solution)
    return solution, res_wrapped.success


def solve_profile_SLSQP(ratios):
    x0 = np.ones(ratios.shape[1])
    bounds = [(min(np.nanmin(ratios), 1/np.nanmax(ratios)), 1) for _ in x0]
    res_wrapped = minimize(triangle_error, args = ratios , x0 = x0, bounds=bounds, method = 'SLSQP', options={'maxiter':10000})
    solution = res_wrapped.x
    solution = solution/np.max(solution)
    return solution, res_wrapped.success


# TRF
def solve_profile_trf(ratios):
    x0 = np.ones(ratios.shape[1])
    bounds = (x0*0+0.01, x0)
    res_wrapped = least_squares(triangle_error, args = [ratios] , x0 = x0*0.5, bounds=bounds, verbose=0, method = 'trf')
    solution = res_wrapped.x
    solution = solution/np.max(solution)
    return solution, res_wrapped.success

## Benchmarking different Optimizers

In [ ]:
from itertools import combinations
from numba.typed import List

n_sequences = 10
n_samples = 6

column_combinations = List()
[column_combinations.append(_) for _ in combinations(range(n_samples), 2)]

methods = {'L-FBGS-B':solve_profile_LFBGSB, 'SLSQP':solve_profile_SLSQP, 'trf':solve_profile_trf}
results = []

for run in range(20):
    signal, sample, ground_truth = generate_dummy_data(n_sequences, n_samples)
    ratios = get_protein_ratios(signal.values, column_combinations)

    for method in methods.keys():
        error = False
        start = time()
        try:
            solution, success = methods[method](ratios)
            mape = np.mean(np.abs((solution-ground_truth)/ground_truth))
        except:
            error = True
            mape = np.nan
            
        
        end = time()

        time_elapsed_s = (end-start)

        results.append((method, time_elapsed_s, mape, error, success))
    
result_df = pd.DataFrame(results, columns=['Method', 'Time Elapsed (s)', 'Mean absolute percentage error', 'Errors', 'Success'])

result_df.groupby('Method').mean()

C:\ProgramData\Anaconda3\envs\alphapept\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
C:\ProgramData\Anaconda3\envs\alphapept\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


,Time Elapsed (s),Mean absolute percentage error,Errors,Success
Method,,,,
L-FBGS-B,0.037563,1.092636,0.0,0.80
SLSQP,0.003630,1.092644,0.0,0.80
trf,0.029791,0.015071,0.2,0.75


> SLSQP seems to be a robust and fast choice

In [ ]:
from itertools import combinations
from numba.typed import List

n_sequences = 10
n_samples = 6

column_combinations = List()
[column_combinations.append(_) for _ in combinations(range(n_samples), 2)]

methods = {'L-FBGS-B':solve_profile_LFBGSB, 'SLSQP':solve_profile_SLSQP, 'trf':solve_profile_trf}
results = []

for run in range(20):
    signal, sample, ground_truth = generate_dummy_data(n_sequences, n_samples)
    ratios = get_protein_ratios(signal.values, column_combinations)

    for method in methods.keys():
        error = False
        start = time()
        try:
            solution, success = methods[method](ratios)
            mape = np.mean(np.abs((solution-ground_truth)/ground_truth))
        except:
            error = True
            mape = np.nan

C:\ProgramData\Anaconda3\envs\alphapept\lib\site-packages\scipy\optimize\optimize.py:282: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "
C:\ProgramData\Anaconda3\envs\alphapept\lib\site-packages\scipy\optimize\_numdiff.py:557: RuntimeWarning: invalid value encountered in subtract
  df = fun(x) - f0


## Wrapper function

In [ ]:
#export
from numba.typed import List
from itertools import combinations

def get_protein_table(df, field = 'int_sum', minimum_ratios = 1, callback = None):
    unique_proteins = df['protein'].unique()
    files = df['shortname'].unique().tolist()
    files.sort()
    
    if len(files) == 1:
        raise ValueError('Only one file present.')
    
    column_combinations = List()
    [column_combinations.append(_) for _ in combinations(range(len(files)), 2)]
        
    columnes_ext = [_+'_LFQ' for _ in files]
    protein_table = pd.DataFrame(index=unique_proteins, columns=columnes_ext + files)

    if field+'_dn' in df.columns:
        field_ = field+'_dn'
    else:
        field_ = field
    
    if df[field_].min() < 0:
        raise ValueError('Negative intensity values present.')
        
    for idx, protein in enumerate(unique_proteins):
        subset = df[df['protein'] == protein].copy()
        per_protein = subset.groupby(['shortname','precursor'])[field_].sum().unstack().T

        for _ in files:
            if _ not in per_protein.columns:
                per_protein[_] = np.nan

        per_protein = per_protein[files]
        per_protein = per_protein.replace(0, np.nan)

        ratios = get_protein_ratios(per_protein.values, column_combinations, minimum_ratios)
        try:
            solution, success = solve_profile_SLSQP(ratios)
        except ValueError:
            logging.info('Normalization with SLSQP failed. Trying BFGS')
            solution, success = solve_profile_LFBGSB(ratios)
            
        file_ids = per_protein.columns.tolist()
        
        pre_lfq = per_protein.sum().values

        if not success or np.sum(~np.isnan(ratios)) == 0: # or np.sum(solution) == len(pre_lfq):
            profile = np.zeros_like(pre_lfq)
            if np.sum(np.isnan(ratios)) != ratios.size:
                logging.info(f'Solver failed for protein {protein} despite available ratios:\n {ratios}')
        else:
            invalid = ((np.nansum(ratios, axis=1) == 0) & (np.nansum(ratios, axis=0) == 0))
            total_int = subset[field_].sum() * solution 
            total_int[invalid] = 0
            profile = total_int * subset[field_].sum().sum() / np.sum(total_int) #Normalize inensity again
            

        protein_table.loc[protein, [_+'_LFQ' for _ in file_ids]] = profile
        protein_table.loc[protein, file_ids] = pre_lfq
        
        if callback:
            callback((idx+1)/len(unique_proteins))
        
    protein_table[protein_table == 0] = np.nan
    protein_table = protein_table.astype('float64')

    return protein_table

def protein_profile(files, minimum_ratios, chunk):
    
    grouped, protein = chunk

    column_combinations = List()
    [column_combinations.append(_) for _ in combinations(range(len(files)), 2)]
    
    selection = grouped.unstack().T.copy()
    selection = selection.replace(0, np.nan)

    if not selection.shape[1] == len(files):
        selection[[_ for _ in files if _ not in selection.columns]] = np.nan

    selection = selection[files]

    ratios = get_protein_ratios(selection.values, column_combinations, minimum_ratios)
    
    try:
        solution, success = solve_profile_SLSQP(ratios)
    except ValueError:
        logging.info('Normalization with SLSQP failed. Trying BFGS')
        solution, success = solve_profile_LFBGSB(ratios)
        
    pre_lfq = selection.sum().values

    if not success or np.sum(~np.isnan(ratios)) == 0: # or np.sum(solution) == len(pre_lfq):
        profile = np.zeros_like(pre_lfq)
        if np.sum(np.isnan(ratios)) != ratios.size:
            logging.info(f'Solver failed for protein {protein} despite available ratios:\n {ratios}')

    else:
        invalid = ((np.nansum(ratios, axis=1) == 0) & (np.nansum(ratios, axis=0) == 0))
        total_int = pre_lfq.sum() * solution 
        total_int[invalid] = 0
        profile = total_int * pre_lfq.sum() / np.sum(total_int) #Normalize inensity again
        
    
    return profile, pre_lfq, protein  


import os
import alphapept.speed
from functools import partial


def protein_profile_parallel(df, minimum_ratios, field, callback=None):
    
    unique_proteins = df['protein'].unique().tolist()
    
    files = df['shortname'].unique().tolist()
    files.sort()
    
    columnes_ext = [_+'_LFQ' for _ in files]
    protein_table = pd.DataFrame(index=unique_proteins, columns=columnes_ext + files)

    grouped = df[[field, 'shortname','precursor','protein']].groupby(['protein','shortname','precursor']).sum()

    column_combinations = List()
    [column_combinations.append(_) for _ in combinations(range(len(files)), 2)]
    
    files = df['shortname'].unique().tolist()
    files.sort()
    
    results = []
    
    if len(files) > 1:
        logging.info('Preparing protein table for parallel processing.')
        split_df = []
        
        for idx, protein in enumerate(unique_proteins):
            split_df.append((grouped.loc[protein], protein))
            if callback:
                callback((idx+1)/len(unique_proteins)*1/5)
        
        results = []

        logging.info(f'Starting protein extraction for {len(split_df)} proteins.')
        with alphapept.speed.AlphaPool(-1) as p:
            max_ = len(split_df)
            for i, _ in enumerate(p.imap_unordered(partial(protein_profile, files, minimum_ratios), split_df)):
                results.append(_)
                if callback:
                    callback((i+1)/max_*4/5+1/5)

        for result in results:
            profile, pre_lfq, protein = result
            protein_table.loc[protein, [_+'_LFQ' for _ in files]] = profile
            protein_table.loc[protein, files] = pre_lfq

        protein_table[protein_table == 0] = np.nan
        protein_table = protein_table.astype('float')
    else:
        protein_table = df.groupby(['protein'])[field].sum().to_frame().reset_index()
        protein_table = protein_table.set_index('protein')
        protein_table.index.name = None
        protein_table.columns=[files[0]] 
        
        if callback:
            callback(1)
        
    return protein_table


def protein_profile_parallel_ap(settings, df, callback=None):
    
    minimum_ratios = settings['quantification']['lfq_minimum_ratio']
    field = settings['quantification']['mode']
    
    if field+'_dn' in df.columns:
        field_ = field+'_dn'
    else:
        field_ = field

    if df[field_].min() < 0:
        raise ValueError('Negative intensity values present.')
        
    protein_table = protein_profile_parallel(df, minimum_ratios, field_, callback)
    
    return protein_table
    

def protein_profile_parallel_mq(evidence_path, protein_groups_path, callback=None):
    logging.info('Loading files')
    
    for file in [evidence_path, protein_groups_path]:
        if not os.path.isfile(file):
            raise FileNotFoundError(f'File {file} not found.')
            
    evd = pd.read_csv(evidence_path, sep='\t') 
    ref = pd.read_csv(protein_groups_path, sep='\t')
            
    experiments = evd['Raw file'].unique().tolist()
    logging.info(f'A total of {len(experiments):,} files.')

    protein_df = []
    
    max_ = len(ref)
    for i in range(max_):
        investigate = ref.iloc[i]
        evd_ids = [int(_) for _ in investigate['Evidence IDs'].split(';')]
        subset = evd.loc[evd_ids].copy()

        subset['protein'] =  investigate['Protein IDs']
        subset['shortname'] = subset['Raw file']
        subset['precursor']  = ['_'.join(_) for _ in zip(subset['Sequence'].values, subset['Charge'].values.astype('str'))]

        protein_df.append(subset)
        
        if callback:
            callback((i+1)/len(ref))
                     
    logging.info(f'A total of {max_:,} proteins.')

    df = pd.concat(protein_df)
    df, normed = delayed_normalization(df, field ='Intensity')
    protein_table = protein_profile_parallel(df, minimum_ratios=1, field='Intensity', callback=callback)
                     
    return protein_table

In [ ]:
if False:
    sample_data = {}

    sample_data['precursor'] = ['Prec_1'] * 2 + ['Prec_2'] * 2 + ['Prec_3'] * 2
    sample_data['shortname'] = ['A','B'] * 3
    sample_data['protein'] = ['X'] * 6
    sample_data['int_sum'] = [0.6, 0.8, 0.6, 1.2, 1.6, 1.2]

    test_df = pd.DataFrame(sample_data)

    display(test_df.head(6))

    protein_profile(test_df, ['A','B'], 'int_sum', 'X')

    profile, pre_lfq, file_ids, protein = protein_profile(test_df, ['A','B'], 'int_sum', 'X')

    print(profile)
    print(pre_lfq)

In [ ]:
if False:
    def test_protein_profile():

        sample_data = {}

        sample_data['precursor'] = ['Prec_1'] * 6 + ['Prec_2'] * 6 + ['Prec_3'] * 6
        sample_data['fraction'] = [1,2,3]*6
        sample_data['shortname'] = ['A','A','A', 'B','B','B'] * 3
        sample_data['protein'] = ['X'] * 18
        sample_data['int_sum'] = [0.6, 0.8, 0.6, 1.2, 1.6, 1.2] * 3

        test_df = pd.DataFrame(sample_data)

        protein_profile(test_df, ['A','B'], 'int_sum', 'X')

        profile, pre_lfq, file_ids, protein = protein_profile(test_df, ['A','B'], 'int_sum', 'X')

        # total intensity should be preserved
        assert np.allclose(profile.sum(), pre_lfq.sum())

        sample_data = {}

        sample_data['precursor'] = ['Prec_1'] * 2 + ['Prec_2'] * 2 + ['Prec_3'] * 2
        sample_data['shortname'] = ['A','B'] * 3
        sample_data['protein'] = ['X'] * 6
        sample_data['int_sum'] = [0.6, 0.8, 0.6, 1.2, 1.6, 1.2]

        test_df = pd.DataFrame(sample_data)

        protein_profile(test_df, ['A','B'], 'int_sum', 'X')

        profile, pre_lfq, file_ids, protein = protein_profile(test_df, ['A','B'], 'int_sum', 'X')

        assert np.allclose(profile.sum(), pre_lfq.sum())

    test_protein_profile()


In [ ]:
#hide
from nbdev.export import *
notebook2script()

Converted 00_settings.ipynb.
Converted 01_chem.ipynb.
Converted 02_io.ipynb.
Converted 03_fasta.ipynb.
Converted 04_feature_finding.ipynb.
Converted 05_search.ipynb.
Converted 06_score.ipynb.
Converted 07_recalibration.ipynb.
Converted 08_quantification.ipynb.
Converted 09_matching.ipynb.
Converted 10_constants.ipynb.
Converted 11_interface.ipynb.
Converted 12_speed.ipynb.
Converted index.ipynb.
